In [45]:
import os
import json
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
TOKEN_LIMIT = 3800

In [16]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def read_csv(filename):
    with open(filename, "r") as f:
        return f.read()

In [57]:
DATA_FOLDER = "/home/js222238/dev/robosuite/data"
DATA_FILE = "task_breakdown_sparse_schema.csv"

In [59]:
## READ DATA
episode_data = read_csv(os.path.join(DATA_FOLDER, DATA_FILE))

In [60]:
episode_data

'"step", "robot0_eef_pos", "cube_pos", "gripper_to_cube_pos", "action", "grasp", "reward"\n1,"[-0.09, -0.03, 1.02]","[-0.02, 0.03, 0.82]","[-0.07, -0.06, 0.2]","[0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -1.0]",-1,0.011\n57,"[-0.09, -0.03, 1.02]","[-0.02, 0.03, 0.82]","[-0.07, -0.06, 0.19]","[3.75, 0.0, 0.0, 0.0, 0.0, -0.0, -1.0]",-1,0.012\n58,"[-0.08, -0.03, 1.01]","[-0.02, 0.03, 0.82]","[-0.06, -0.06, 0.19]","[0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -1.0]",-1,0.013\n67,"[-0.08, -0.03, 1.01]","[-0.02, 0.03, 0.82]","[-0.06, -0.06, 0.19]","[3.75, 0.0, 0.0, 0.0, 0.0, -0.0, -1.0]",-1,0.013\n68,"[-0.07, -0.03, 1.01]","[-0.02, 0.03, 0.82]","[-0.05, -0.06, 0.19]","[0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -1.0]",-1,0.014\n83,"[-0.06, -0.03, 1.01]","[-0.02, 0.03, 0.82]","[-0.04, -0.06, 0.19]","[3.75, 0.0, 0.0, 0.0, 0.0, -0.0, -1.0]",-1,0.014\n84,"[-0.06, -0.03, 1.01]","[-0.02, 0.03, 0.82]","[-0.04, -0.06, 0.19]","[0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -1.0]",-1,0.015\n95,"[-0.06, -0.03, 1.01]","[-0.02, 0.03, 0.82]","[-0.04, -0.0

In [61]:
## GENERAT PROMPT
schema = "{{step, robot0_eef_pos, cube_pos, gripper_to_cube_pos, action, grasp, reward}}"


task_description = f"""
Your task is to ingest the following data and describe what occurred during a robot episode.
Categorize the data into subtasks. Create substasks and their corresponding labels based on what you think is occuring during each subtask. Use the following format:
```
Subtask: <subtask name>
Description: <description of subtask>
Steps: <the step numbers that correspond to the subtask>
```
"""

data_description = """
The data captures a simulated episode of a robot end effector manipulating an environment. Each entry in the schema contains an observation. There is missing data because the step numbers are only when there is a change in action. Note that you need to incorporate the 'previous_output' into your description of the current step.
"""

schema_description = f"""
Each observation has the schema: {schema}
"""

previous_output = "No previous output, this is the first step."

def get_prompt(data_chunk: str, previous_output: str) -> str:
    return """{task_description}\n
    {data_description}\n
    {schema_description}\n
    previous_output: {previous_output}\n
    Data: {data_chunk}
    """

In [62]:
def parse_data(data):
    "Parse the csv file into data chunks smaller than the token limit"
    data_chunks = []
    data_chunk = ""
    for line in data.split("\n"):
        if len(data_chunk) + len(line) < TOKEN_LIMIT:
            data_chunk += line + "\n"
        else:
            data_chunks.append(data_chunk)
            data_chunk = ""
    return data_chunks

In [63]:
data_chunks = parse_data(episode_data)

In [64]:
for data_chunk in data_chunks:
    prompt = get_prompt(data_chunk, previous_output)
    previous_output = get_completion(prompt)
    print(previous_output)

KeyboardInterrupt: 

In [33]:
response = get_completion(prompt)

In [35]:
print(response)

Subtask: Robot approaching the cube
Description: The robot is moving towards the cube to get closer to it.
Steps: 1-58

Subtask: Robot adjusting position
Description: The robot is making small adjustments to its position.
Steps: 58-84

Subtask: Robot grasping the cube
Description: The robot is attempting to grasp the cube.
Steps: 84-96

Subtask: Robot lifting the cube
Description: The robot is lifting the cube off the surface.
Steps: 96-116

Subtask: Robot moving the cube
Description: The robot is moving the cube to a different location.
Steps: 116-140

Subtask: Robot releasing the cube
Description: The robot is releasing the cube.
Steps: 140-163

Subtask: Robot moving away from the cube
Description: The robot is moving away from the cube.
Steps: 163-374

Subtask: Robot adjusting position
Description: The robot is making small adjustments to its position.
Steps: 374-402

Subtask: Robot moving the cube
Description: The robot is moving the cube to a different location.
Steps: 402-430

Su

In [38]:
response=get_completion("are you aware of what just in the previuos message?")
print(response)

I apologize, but I cannot provide a response as I do not have access to the previous message.
